In [1]:
from __future__ import print_function, division, absolute_import

#Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('../mnist', one_hot=True)

import tensorflow as tf
from keras import backend as K

#Clear TF memory
cfg = K.tf.ConfigProto()
cfg.gpu_options.allow_growth = True
K.set_session(K.tf.Session(config=cfg))

C:\Users\StrikeWade\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting ../mnist\train-images-idx3-ubyte.gz
Extracting ../mnist\train-labels-idx1-ubyte.gz
Extracting ../mnist\t10k-images-idx3-ubyte.gz
Extracting ../mnist\t10k-labels-idx1-ubyte.gz


Using TensorFlow backend.


In [2]:
#Parmeters
learning_rate = 0.01
num_steps = 1000
batch_size = 128
display_step = 100

#Network Parameters
n_input = 784
n_classes = 10
dropout = 0.75 #probability to keep units

sess = tf.Session()

#Create a tensor dataset from the images and the labels
dataset = tf.contrib.data.Dataset.from_tensor_slices((mnist.train.images, mnist.train.labels))
print(dataset)
print(' ')

#Create batches of data
dataset = dataset.batch(batch_size)
print(dataset)

#Create an iterator to go over all the dataset
iterator = dataset.make_initializable_iterator()

#It is better to use 2 placeholders, to avoid to load all data into memory,
#and avoid the 2gb restriction length of a tensor
x = tf.placeholder(dtype=tf.float32, shape=[None, n_input])
y = tf.placeholder(dtype=tf.float32, shape=[None, n_classes])

#Init the iterator
sess.run(iterator.initializer, feed_dict={x: mnist.train.images, y: mnist.train.labels})

#Neural net input
X, Y = iterator.get_next()

Instructions for updating:
Use `tf.data.Dataset.from_tensor_slices()`.
<Dataset shapes: ((784,), (10,)), types: (tf.float32, tf.float64)>
 
<Dataset shapes: ((?, 784), (?, 10)), types: (tf.float32, tf.float64)>


In [3]:
X

<tf.Tensor 'IteratorGetNext:0' shape=(?, 784) dtype=float32>

In [4]:
Y

<tf.Tensor 'IteratorGetNext:1' shape=(?, 10) dtype=float64>

In [5]:
#Create a classic CNN
#Create model
def conv_net(x, n_classes, dropout, reuse, is_training):
    #Define a scope for reusing the variables
    with tf.variable_scope('ConvNet', reuse=reuse):
        #MNIST data input is a 1-D vector of 784 features(28*28)
        #Reshape to match picture format [height * width * channel]
        #Tensor input become 4-D [batch size * height * width * channel]
        x = tf.reshape(tensor=x, shape=[-1, 28, 28, 1])
        
        #Convolution layer 1 with 32 filters and kernel of 5
        conv1 = tf.layers.conv2d(inputs=x, filters=32, kernel_size=5, activation=tf.nn.relu)
        #MaxPooling with strides of 2 and kernel size of 2
        conv1 = tf.layers.max_pooling2d(inputs=conv1, strides=2, pool_size=2)
        
        #Convolution layer 2 with 64 filters and kernel of 3
        conv2 = tf.layers.conv2d(inputs=conv1, filters=64, kernel_size=3, activation=tf.nn.relu)
        #Maxpooling with strides of 2 and kernel size of 2
        conv2 = tf.layers.max_pooling2d(inputs=conv2, strides=2, pool_size=2)
        
        #Flatten the data to a 1-D vector for the fully connected layer
        fc1 = tf.contrib.layers.flatten(conv2)
        
        #Fully connected layer
        fc1 = tf.layers.dense(inputs=fc1, units=1024)
        #Apply dropout
        fc1 = tf.layers.dropout(inputs=fc1, rate=dropout, training=is_training)
        
        #Output layer, class prediction
        out = tf.layers.dense(inputs=fc1, units=n_classes)
        #Becasue softmax_cross_entropy_with_logits already apply softmax,
        #we only apply softmax to testing network
        out = tf.nn.softmax(out) if not is_training else out
    return out

In [6]:
#Create graph for training
logits_train = conv_net(x=X, n_classes=n_classes, dropout=dropout, reuse=False, is_training=True)
#Create graph for testing (not apply dropout)
logits_test = conv_net(x=X, n_classes=n_classes, dropout=dropout, reuse=True, is_training=False)

#Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits_train, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

#Evaluate model(with test logits)
correct_pred = tf.equal(tf.argmax(logits_test, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, dtype=tf.float32))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [7]:
init = tf.global_variables_initializer()

sess.run(init)

#Training
for step in range(1, num_steps+1):
    
    try:
        #Run optimization
        sess.run(train_op)
    except tf.errors.OutOfRangeError:
        #Reload the iterator when it reaches the end of the dataset
        sess.run(iterator.initializer, feed_dict={x: mnist.train.images, y: mnist.train.labels})
        sess.run(train_op)
        
    if step % display_step == 0 or step == 1:
        loss, acc = sess.run([loss_op, accuracy])
        print('Step: %d, Minibatch Loss= %.4f, Training Accuracy= %.4f' % (step, loss, acc))

print('Training Completed!!!')

Step: 1, Minibatch Loss= 8.9890, Training Accuracy= 0.0859
Step: 100, Minibatch Loss= 0.2783, Training Accuracy= 0.9297
Step: 200, Minibatch Loss= 0.2864, Training Accuracy= 0.9297
Step: 300, Minibatch Loss= 0.2681, Training Accuracy= 0.9453
Step: 400, Minibatch Loss= 0.1516, Training Accuracy= 0.9453
Step: 500, Minibatch Loss= 0.2090, Training Accuracy= 0.9531
Step: 600, Minibatch Loss= 0.2018, Training Accuracy= 0.9453
Step: 700, Minibatch Loss= 0.1181, Training Accuracy= 0.9844
Step: 800, Minibatch Loss= 0.1918, Training Accuracy= 0.9766
Step: 900, Minibatch Loss= 0.3203, Training Accuracy= 0.9297
Step: 1000, Minibatch Loss= 0.1445, Training Accuracy= 0.9766
Training Completed!!!
